<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/test/NLP_Test_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test y Analisis de datos de entrada

### Clonamos repo

In [ ]:
from google.colab import userdata

git_token = userdata.get('GIT_TOKEN')
git_username = "gabrielfernandorey"
git_repository = "ITBA-NLP.git"

!git clone https://{git_token}@github.com/{git_username}/{git_repository}

### Instalacion de librerias

In [ ]:
!pip install bertopic spacy
!python -m spacy download es_core_news_lg

In [ ]:
!pip install keybert[spacy]

### Librerias

In [5]:
import pandas as pd
import numpy as np
import re, os
from datetime import datetime
from dateutil.parser import parse
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

In [ ]:
import spacy
from bertopic import BERTopic

### Path

In [6]:
PATH=os.environ.get('PATH_LOCAL', '/content/ITBA-NLP/data/')
PATH

'/content/ITBA-NLP/data/'

### Data

In [8]:
# Read the parquet file
df_parquet = pd.read_parquet(PATH+'df_joined_2024-04-01 00_00_00.parquet')
df_parquet.head(1)

,Asset Name,Author Id,Author Name,Keyword Id,Keyword Name,Entity Id,Entity Name,Media Group Id,Media Group Name,Impact,...,in__text,out__entities,out__potential_entities,predicted_at_entities,out__keywords_sorted,predicted_at_keywords,start_time_utc,start_time_local,truncated_text,title_and_text
Asset Id,,,,,,,,,,,,,,,,,,,,,
105628101,Elecciones en Venezuela: María Corina Machado ...,36192,Infobae,1932002 | 417739 | 1687638 | 36187 | 7476 | 50...,"[falsas conspiraciones armadas, sustituta, det...",219925 | 210613 | 219770 | 36424 | 1129437,"[Nicolás Maduro, Jorge Rodríguez, Marcelo Ebra...",0,,7406333,...,Fotografía de archivo de la líder antichavista...,"[Nicolás Maduro, Marcelo Ebrard, Jorge Rodrígu...","[Jorge Rodríguez, Nicolás Maduro, Rayner Peña ...",2024-04-02 08:11:57.825777,"[elecciones presidenciales, candidatura presid...",2024-04-02 08:17:44.372891+00:00,2024-04-02,2024-04-01 21:00:00,Fotografía de archivo de la líder antichavista...,Elecciones en Venezuela: María Corina Machado ...


In [9]:
# Solo el titulo de la noticia
data = list(df_parquet['in__title'])
data[0]

'Elecciones en Venezuela: María Corina Machado pidió más apoyo de Noruega para garantizar unas presidenciales libres'

In [10]:
len(data)

3104

In [ ]:
# Todo el cuerpo de la noticia
data = list(df_parquet['in__text'])
data[0]

### NER
Se busca comparar las entidades de cada documento provistas en el dataset vs la generación de nuevas entidades de ese documento con Spacy

In [ ]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

In [ ]:
%%time
# Detectar entidades en el primer documento usando spaCy
entidades_spa_one = []
for doc in data:
    procesado = spa(doc)
    entidades_spa_one.append([(ent.text, ent.label_) for ent in procesado.ents])
    break

In [ ]:
entidades_spa_one

In [ ]:
# Texto original primer documento
text = df_parquet.iloc[0]['in__text']
pprint.pp(text)

Extracción de entidades unicas ordenadas de mayor a menor por cantidad de repeticiones.

In [ ]:
# Aplanar la lista de listas
datos_flat = [item for sublist in entidades_spa_one for item in sublist]

# Contar las repeticiones de cada entidad
contador = Counter(datos_flat)

# Ordenar por cantidad de repeticiones de mayor a menor
ordenado = sorted(contador.items(), key=lambda x: x[1], reverse=True)

# Eliminar duplicados y mantener ordenado por cantidad de repeticiones
resultado = [(entidad, tipo) for (entidad, tipo), _ in ordenado]

# Imprimir resultados
for entidad, tipo in resultado:
    print(f"{entidad} ({tipo}) - {contador[(entidad, tipo)]} veces")

In [ ]:
# Entidades obtenidas con modelo lg de spacy ( simplificado )
[ k for k, v  in resultado if len(k.split()) <= 4 ]

In [ ]:
# Entidades originales del datatset
df_parquet.iloc[0]['out__entities']


In [ ]:
# Entidades originales del datatset potenciales
df_parquet.iloc[0]['out__potential_entities']

Obtener entidades para todas las noticias

In [ ]:
%%time
# Detectar entidades para todos los documentos usando spaCy
entidades_spa = []
for doc in data:
    procesado = spa(doc)
    entidades_spa.append([(ent.text, ent.label_) for ent in procesado.ents])

In [ ]:
entidades_spa[0]

### NER - Combinando Spacy + BERTopic

In [ ]:
%%time
# Extraccion de entidades en todos los documentoa usando spaCy
entidades_spa = []
for doc in data:
    procesado = spa(doc)
    entidades_spa.append([(ent.text, ent.label_) for ent in procesado.ents])


In [ ]:
# Obtenemos las entidades unicas ordenadas de mayor a menor por cantidad de repeticiones
# para todos los documentos

entidades_spa_ber = []
for ent in entidades_spa:

    # Contamos entidades
    entity_counts = Counter(item[0] for item in ent)

    # Ordenamos de mayor a menor
    ordenado = sorted(entity_counts.items(), key=lambda x: x[1], reverse=True)

    # Extraemos solo las entidades
    resultado = [entidad for entidad, _ in ordenado]

    entidades_spa_ber.append(resultado)

In [ ]:
entidades_spa_ber[0]

In [ ]:
# Instanciamos modelo
topic_model = BERTopic(
        language='spanish',
        calculate_probabilities=True
)

In [ ]:
%% time
topics, probs = topic_model.fit_transform(data)

In [ ]:
# el topico -1 contiene documentos de descarte ( sin topico )
print(f"{len(set(topics))} tópicos, (incluye el -1)")

In [ ]:
topic_model.visualize_barchart(top_n_topics=len(set(topics))-1, width=400)

In [ ]:
# Mostrar los temas descubiertos
print("Información de los temas descubiertos:")
print(topic_model.get_topic_info())

Analizamos el primer documento y su relacion con el topico

In [ ]:
# Elegimos el primer documento. Cual es el tópico del primer documento?
x_topic = topics[0]
x_topic

In [ ]:
# De que tema trata el topico del documento elegido?
topic_model.get_topic_info(x_topic)

In [ ]:
# Cuales son las primeras 10 entidades descubiertas para el primer documento elegido?
entidades_spa_ber[0][:10]

In [ ]:
# Combinar la información de temas y entidades
tema_entidades = {}
for idx, topic in enumerate(topics):
    if topic not in tema_entidades:
        tema_entidades[topic] = []
    tema_entidades[topic].extend(entidades_spa_ber[idx])

In [ ]:
# Visualizar para el tópico del documento elegido, las entidades de su documento,
# más las entidades de otros documentos que comparten el mismo topico
tema_entidades[x_topic]

### Obtenemos las entidades de un topico
( a partir de todas las entidades de los documentos de un topico en particular )

In [ ]:
# Contar las repeticiones de cada entidad
contador_topic = Counter(tema_entidades[x_topic])

# Ordenar por cantidad de repeticiones de mayor a menor
ordenado_topic = sorted(contador_topic.items(), key=lambda x: x[1], reverse=True)

# Imprimir resultados
c = 0
for entidad in ordenado_topic:
    print(f"{entidad}")
    c += 1
    if c == 10:
      break

In [ ]:
# Cantidad de noticias por topico
T = topic_model.get_document_info(data)
docs_per_topics = T.groupby(["Topic"]).apply(lambda x: x.index).to_dict()

In [ ]:
# Noticias relacionadas al topico del documento elegido
docs_per_topics[x_topic]

In [ ]:
data[0][:150].strip()

In [ ]:
# Visualizar un fragmento de cada una de las noticias del topico del documento elegido
for doc in docs_per_topics[x_topic]:
    linea = data[doc][:150].replace("\n", "").strip()
    print(f"{doc}: {linea}\n")


In [ ]:
# Encontrar el topico de una noticia del dataset

doc = np.random.randint(len(data))
doc = 0
for k,v in docs_per_topics.items():
  if doc in v:
    print(f"Noticia nro: {doc}")
    print(data[doc][:150].strip())
    print(f"Entidades originales: {df_parquet.iloc[doc]['out__entities']}")
    print(f"Nuevas Entidades: {entidades_spa_ber[doc]}")
    print(f"Topico: {k}")
    print("Keywords:")
    for k,v in topic_model.get_topic(k):
      print(f"    {k}:\t {v}")


### Utilizando NER Transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

In [ ]:
model_ner = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

In [ ]:
%%time
# Detectar entidades para todos los documento usando transformers
entidades_trf = []
for doc in data:
    procesado = model_ner(doc)
    entidades_trf.append([ent["word"] for ent in procesado])


In [ ]:
entidades_trf[0]

In [ ]:
from collections import Counter


list

In [ ]:
bigrams = [
    [('El', 'DET'), ('argentino', 'NOUN')],
    [('ahorro', 'NOUN'), ('interno', 'ADJ')],
    [('interno', 'ADJ'), ('que', 'SCONJ')],
    [('que', 'SCONJ'), ('su', 'DET')],
    [('suficiente', 'ADJ'), ('ahorro', 'NOUN')],
    [('ahorro', 'NOUN'), ('interno', 'ADJ')],
    [('de', 'ADP'), ('ahorro', 'NOUN')],
    [('ahorro', 'NOUN'), ('es', 'AUX')],
    [('es', 'AUX'), ('el', 'DET')],
    [('el', 'DET'), ('dólar', 'NOUN')]
]

In [ ]:
# return the most frequent words that appear next to a particular keyword
def get_neighbor_words(keyword, bigrams, pos_label = None):

    neighbor_words = []
    keyword = keyword.lower()

    for bigram in bigrams:

        #Extract just the lowercased words (not the labels) for each bigram
        words = [word.lower() for word, label in bigram]

        #Check to see if keyword is in the bigram
        if keyword in words:
            idx = words.index(keyword)
            for word, label in bigram:

                #Now focus on the neighbor word, not the keyword
                if word.lower() != keyword:
                    #If the neighbor word matches the right pos_label, append it to the master list
                    if label == pos_label or pos_label == None:
                        if idx == 0:
                            neighbor_words.append(" ".join([keyword, word.lower()]))
                        else:
                            neighbor_words.append(" ".join([word.lower(), keyword]))

    return Counter(neighbor_words).most_common()



In [ ]:
get_neighbor_words("ahorro", bigrams, pos_label='ADJ')

[('ahorro interno', 2), ('suficiente ahorro', 1)]

### NER - Utilizamos KeyBERT

In [11]:
from keybert import KeyBERT

In [12]:
kw_model = KeyBERT()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
def extract_keybert(ID):
    doc = df_parquet['in__text'][df_parquet.index == ID].values[0]
    keywords = kw_model.extract_keywords(doc)
    print("Keywords (KeyBERT):", [k for k,v in keywords], "\n")
    print("Keywords (df):", df_parquet['Keyword Name'][df_parquet.index == ID].values[0], "\n")
    print("Keywords (df out sorted):", df_parquet['out__keywords_sorted'][df_parquet.index == ID].values[0])

    return

In [23]:
extract_keybert(105628101)

Keywords (KeyBERT): ['oposición', 'elecciones', 'venezuela', 'delegación', 'calificado'] 

Keywords (df): ['falsas conspiraciones armadas' 'sustituta' 'detención injustificada'
 'régimen' 'inhabilitación' 'apoyo internacional'
 'candidatura presidencial' 'elecciones presidenciales'] 

Keywords (df out sorted): ['elecciones presidenciales' 'candidatura presidencial'
 'apoyo internacional' 'inhabilitación' 'régimen'
 'detención injustificada' 'sustituta' 'falsas conspiraciones armadas'
 'sucesora' 'impedimentos' 'líder opositora' 'jefe' 'dictador'
 'próximas desapariciones' 'retirada' 'partido' 'obstáculos' 'violaciones'
 'alusión' 'equipo' 'listado' 'canciller' 'garantías electorales'
 'delegación' 'fotografía' 'negociación' 'negociaciones' 'conversaciones'
 'tiempo' 'miembro' 'alcance' 'país caribeño' 'corte' 'nombres'
 'actores políticos' 'candidatos' 'representante' 'mesa' 'pruebas' 'foto'
 'compañeros' 'obstante' 'plataforma' 'disposición' 'pueblo venezolano'
 'ambas partes' 'comuni

In [21]:
df_parquet['in__text'][df_parquet.index == 105628101].values[0]

'Fotografía de archivo de la líder antichavista María Corina Machado (EFE/ Rayner Peña R.)\n\nLa líder opositora María Corina Machado pidió a Noruega -país mediador en las negociaciones entre el régimen de Venezuela y la oposición mayoritaria- extremar su apoyo para garantizar que en el país caribeño se celebren, el próximo 28 de julio, unas presidenciales “libres y justas”, según una comunicación difundida este lunes, que insta a que presione al dictador Nicolás Maduro para que cumpla con el Acuerdo de Barbados y admita la candidatura presidencial de su sucesora, Corina Yoris.\n\nLa carta, dirigida al primer ministro noruego, Jonas Gahr, tiene el objetivo de exhortar a su Gobierno, en “su calidad de facilitador del proceso de diálogo y negociación entre actores políticos de Venezuela, a extremar los recursos diplomáticos a su disposición” para que el acuerdo de Barbados -firmado por ambas partes en octubre, que establece garantías electorales- sea “cumplido integralmente”.\n\nMachado,